In [1]:
#Cargar la extensión autoreload
%load_ext autoreload
%autoreload 2

# Document Processing tools
#from libs.loaders.postgres_loader import PostgresLoader
import re
from libs.loaders.postgres_loader import PostgresLoader
#from libs.splitters.basic_text_splitter import BasicTextSplitter
from libs.storage.basic_document_storage import BasicStorage
# Retriever tools
from libs.retrieval.vector_retriever import VectorRetriever
from libs.retrieval.lexical_retriever import LexicalRetriever
from libs.retrieval.hybrid_retriever import HybridRetriever
# The Agent
#from libs.agent.chatbot import Chatbot

# Prompt Tools
#from libs.prompts.academic_assistant_prompt import system_prompt as academic_system_prompt
#from libs.prompts.academic_assistant_prompt import user_prompt as academic_user_prompt

In [3]:
# 1. Load the documents and chunk them
# Load the documents
#loader = PDFLoader(file_filter={".pdf"})
loader = PostgresLoader()
documents = loader.load_data()
# Split the documents in chunks
#splitter = BasicTextSplitter()
#chunked_documents = splitter.split_documents(documents)

In [9]:
def filter_documents(data_filter,data):
    results=[]
    #filter_keys=["name","organism","state","year","normtype","number","gazette","slug"]
    for doc in data:
        match = True
        for key,value in data_filter.items():
            if doc[key]!=value:
                match = False
                break
            
        if match:
            results.append(doc)
    
    return results
        

In [25]:
def find_matches_with_positions(text):
    # Use re.finditer to get all matches
    pattern = r"_+"
    matches = re.finditer(pattern, text)
    
    # Iterate through the matches and extract the information
    positions = []
    for match in matches:
        start = match.start()  # Start index of the match
        end = match.end()      # End index of the match
        substring = match.group()  # Substring that matches the pattern
        positions.append((substring, start, end))
    
     # Print the results
    #for substring, start, end in positions:
    #    print(f"Substring: '{substring}', Start: {start}, End: {end}")
    
    if len(positions)>= 2:
        return text[positions[0][2]:positions[1][1]]
    
    elif len(positions)==1:
        return text[:positions[0][1]]
    
    else:
        return text
    
    
    # return positions

In [26]:
new_docs = []
for doc in documents:
    new_doc = doc.copy()
    new_doc['text']= find_matches_with_positions(doc['text'])
    new_docs.append(new_doc)

In [56]:
fail = []
for doc in new_docs:
    if doc['text']=='':
        fail.append(doc)
    

In [58]:
filterded_docs = []
for doc in fail:
    filterded_docs.append(filter_documents({"id":doc['id']},documents)[0])

In [59]:
docs_with_text = []
for doc in filterded_docs:
    if doc['text']!='':
        docs_with_text.append(doc)

In [70]:
docs_with_text[1600]

{'id': '960d07028b2006938941db24781d3cf65e1d9347',
 'name': 'Resolución 127 de 2024 de Ministerio de Energía y Minas',
 'text': '1795\nGaceta Oficial de la República\n20/11/2024\nGOC-2024-O111\nPOR CUANTO: La Empresa de la Sal, en forma abreviada ENSAL, ha solicitado el \ntraspaso de la concesión de explotación y procesamiento que señalada en el Por Cuanto \nanterior, a favor de la Empresa Filial Salinera de Guantánamo.\nPOR TANTO: En el ejercicio de las atribuciones que me han sido conferidas en el \nArtículo 145, inciso e), de la Constitución de la República de Cuba, \nRESUELVO\nPRIMERO: Traspasar la concesión de explotación y procesamiento en el área denominada \nSalinas Cerro Guayabo-Caimanera, ubicada en el municipio de Caimanera de la \nprovincia de Guantánamo, a favor de la Empresa Filial Salinera de Guantánamo.\nSEGUNDO: El traspaso que se autoriza está vigente hasta el 26 de octubre de 2049.\nTERCERO: Los términos, condiciones y obligaciones dispuestos en  las resoluciones 91,

In [24]:
# 2. Create a retriever to handle the retrieval process.
# Store the documents
basic_storage = BasicStorage()
# For handling the storage and retrieval of data chunks with semantic search
semantic_retriever = VectorRetriever()
# For handling the storage and retrieval of data chunks with lexical search
lexical_retriever = LexicalRetriever()
# A retriever for handling all the retrieval process
hybrid_retriever = HybridRetriever(basic_storage, retrievers=[semantic_retriever,lexical_retriever])
# Add the documents to the retriever
print("Adding documents to the retriever")
hybrid_retriever.add(chunked_documents)


Adding documents to the retriever


WARNING clustering 659 points to 25 centroids: please provide at least 975 training points


BM25S Create Vocab:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Convert tokens to indices:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/659 [00:00<?, ?it/s]

In [25]:
query1 = "Cuales son los subsistemas endogenos del modelo pentagonal"
docu = hybrid_retriever.search([query1])

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

['1-124', '1-13', '1-262', '1-60', '1-61', '1-64', '1-80', '2-10', '2-11', '2-15', '2-169', '2-239', '2-243', '2-244', '2-249', '2-34', '2-60', '2-69', '2-80', '2-95']
['1-124', '1-13', '1-262', '1-60', '1-61', '1-64', '1-80', '2-10', '2-11', '2-15', '2-169', '2-239', '2-243', '2-244', '2-249', '2-34', '2-60', '2-69', '2-80', '2-95']


In [31]:
print(docu[2])

{'id': '2-60', 'text': 'sea el modelo que se siga, en todas las sociedades modernas se delimita, se legaliza, y se conduce estatalmente el proceso económico. Es usual que los gober nantes aseguren el equilibrio de la moneda, la política comercial, la estabilidad de los pr ecios, el control de las inversiones, asignen 49BIDART CAMPOS Germán J. La Constitución económica . Fundación Universitaria de Derecho, Administración y Política, México, 2003, p. 37 48 recursos por vía presupuestaria, establezc an la política tributaria, dicten medidas promocionales, etc., y toda esa actividad económica, su estructura, organización, funcionamiento es comúnmente regulada, porque aunque su contenido a simple vista pudiera centrarse principalmente en los recursos con que se cuenta; es vital para el logro de sus fines, tener en cuenta cómo se permiten usar esos recursos, precisamente, en virtud de la estrecha y recíproca relación que existe entre los fenómenos jurídicos y los económicos. 2º En ese sentid

In [49]:
bot = Chatbot(hybrid_retriever,
              system_prompt=academic_system_prompt,
              user_prompt=academic_user_prompt
              )

In [54]:
query1 = "Cuales son los subsistemas endogenos del modelo pentagonal"

In [55]:
bot.reply(query1)

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

'Los subsistemas endógenos del modelo pentagonal son: recursos turísticos y atractivos, productos turísticos, organización y comercialización de la oferta turística, y flujos turísticos.'

In [52]:
query2 = "Menciona todos los subsistemas que encuentres en el contexto del modelo pentagonal"

In [53]:
bot.reply(query2)

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

'Subsistemas exógenos o integrantes del ‘entorno turístico’, Subsistemas endógenos o propiamente turísticos, Subsistema macroentorno o ambiente general'